<a href="https://colab.research.google.com/github/bruAristimunha/Across-Modalities/blob/master/baseline_methods_grilo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne pyriemann monai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pathlib import Path

import mne
from tqdm import tqdm

from numpy import array, float32, float64, savez

from mne.decoding import CSP as mneCSP, Vectorizer

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.svm import SVC

from pyriemann.classification import MDM
from pyriemann.estimation import Covariances, ERPCovariances, XdawnCovariances
from pyriemann.spatialfilters import CSP, Xdawn
from pyriemann.tangentspace import TangentSpace

from monai.utils import set_determinism

mne.set_log_level("CRITICAL")

In [ ]:
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    balanced_accuracy_score,
    cohen_kappa_score,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
    roc_auc_score,
)


def true_positive(y_true, y_pred):
    """
    Returning true positive.
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    return confusion_matrix(y_true, y_pred)[0, 0]


def true_negative(y_true, y_pred):
    """
    Returning true negative.
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    return confusion_matrix(y_true, y_pred)[1, 1]


def false_positive(y_true, y_pred):
    """
    Returning false positive.
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    return confusion_matrix(y_true, y_pred)[1, 0]


def false_negative(y_true, y_pred):
    """
    Returning false negative.
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    return confusion_matrix(y_true, y_pred)[0, 1]


def tp_n(y_true, y_pred):
    """
    True Positive Normalized
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    conf_ma = confusion_matrix(y_true, y_pred)
    conf_ma = conf_ma.astype("float") / conf_ma.sum(axis=1)[:, np.newaxis]
    return conf_ma[0, 0]


def tn_n(y_true, y_pred):
    """
    True Negative Normalized
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    conf_ma = confusion_matrix(y_true, y_pred)
    conf_ma = conf_ma.astype("float") / conf_ma.sum(axis=1)[:, np.newaxis]
    return conf_ma[1, 1]


def fp_n(y_true, y_pred):
    """
    False Positive Normalized
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    conf_ma = confusion_matrix(y_true, y_pred)
    conf_ma = conf_ma.astype("float") / conf_ma.sum(axis=1)[:, np.newaxis]
    return conf_ma[1, 0]


def fn_n(y_true, y_pred):
    """
    False Negative Normalized
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    """
    conf_ma = confusion_matrix(y_true, y_pred)
    conf_ma = conf_ma.astype("float") / conf_ma.sum(axis=1)[:, np.newaxis]
    return conf_ma[0, 1]


scoring = {
    "acc": make_scorer(accuracy_score),
    "kappa": make_scorer(cohen_kappa_score),
    "average_precision": make_scorer(average_precision_score),
    "balanced_accuracy": make_scorer(balanced_accuracy_score),
    "roc_auc": make_scorer(roc_auc_score),
    "f1": make_scorer(f1_score),
    "recall_binary": make_scorer(recall_score),
    "f1_weighted": make_scorer(f1_score, average="weighted"),
    "precision": make_scorer(precision_score, average="weighted"),
    "recall": make_scorer(recall_score, average="weighted"),
    "tp": make_scorer(true_positive),
    "tn": make_scorer(true_positive),
    "fp": make_scorer(false_positive),
    "fn": make_scorer(false_negative),
    "tp_n": make_scorer(tp_n),
    "tn_n": make_scorer(tn_n),
    "fp_n": make_scorer(fp_n),
    "fn_n": make_scorer(fn_n),
}


In [4]:
"""
Authors: Bruno Aristimunha <b.aristimunha@gmail.com>

Baseline script to analyse the EEG Dataset.

"""
seed=46
path_output="."
n_cv=5
n_jobs=7
experiment_name="baseline_emotions"

# Setting run information

set_determinism(seed=seed)

output_dir = Path(f"{path_output}/runs/")
output_dir.mkdir(exist_ok=True, parents=True)

experiment_name = (
    experiment_name + "-baseline"
)

run_dir = output_dir / (experiment_name)
print(f"The run_dir is {run_dir}")
if run_dir.exists():
    pass
else:
    print("First time running, creating folder")
    run_dir.mkdir(exist_ok=True, parents=True)

###################################################################

# Code to read your dataset
# You will need to replace with part
# Here we are creating fake EEG data.

fs=250
n_trial=1152
n_channels=22

signal_len=4 #in seconds
n_points = fs * signal_len
t = np.arange(n_points) / fs
x_eeg = np.random.rand(n_trial, n_channels, n_points)
y_label = np.random.choice([0, 1], size=(n_trial,))


###################################################################


# build your pipeline
pipelines = {}

pipelines["Dummy"] = make_pipeline(
    DummyClassifier(
        strategy="most_frequent", random_state=seed
    )
)

pipelines["Vect_LR"] = make_pipeline(
    Vectorizer(), StandardScaler(), LogisticRegression()
)

pipelines["ERPCov_TS"] = make_pipeline(
    ERPCovariances(estimator="oas"), TangentSpace(), LogisticRegression()
)
pipelines["ERPCov_MDM"] = make_pipeline(
    ERPCovariances(estimator="oas"), MDM()
)
pipelines["XdawnCov_TS"] = make_pipeline(
    XdawnCovariances(estimator="oas"), TangentSpace(), LogisticRegression()
)
pipelines["XdawnCov_MDM"] = make_pipeline(
    XdawnCovariances(estimator="oas"), MDM()
)
pipelines["Xdawn_RegLDA"] = make_pipeline(
    Xdawn(2, classes=[1]),
    Vectorizer(),
    LDA(shrinkage="auto", solver="eigen"),
)

pipelines["TGSP_SVM"] = make_pipeline(
    Covariances("oas"),
    TangentSpace(metric="riemann"),
    SVC(kernel="linear"),
)

pipelines["MDM"] = make_pipeline(Covariances("oas"), 
                                 MDM(metric="riemann"))
pipelines["CO_CSP_LDA"] = make_pipeline(
    Covariances("oas"), CSP(nfilter=4), LDA()
)
pipelines["CSP_LOGISTICREG"] = make_pipeline(
    mneCSP(), LogisticRegression()
)
pipelines["CSP_LDA"] = make_pipeline(mneCSP(), LDA())

# cross validation
with tqdm(pipelines) as pbar:
  for name in pbar:
      file_save_name = experiment_name +"-"+ name
      pbar.set_description(f"Processing method {name}")

      if name.upper() in ["CSP_LDA", "CSP_LOGISTICREG"]:

          x_eeg = x_eeg.astype(float64)
      else:
          x_eeg = x_eeg.astype(float32)


      metrics = cross_validate(
          pipelines[name],
          x_eeg,
          y_label,
          cv=n_cv,
          scoring=scoring,
          n_jobs=n_jobs,
      )

      savez(
          run_dir / (file_save_name + ".npz"),
          metrics=metrics,
          model_name=name.lower(),
      )


print("---------------------------------------")

The run_dir is runs/baseline_emotions-baseline


Processing method CSP_LDA: 100%|██████████| 12/12 [02:29<00:00, 12.44s/it]

---------------------------------------
